In [ ]:
# バッファ内のデータのみを可視化する
# 可視化する特徴量は，データ拡張を加えたデータの平均
def feature_umap_v4(model, train_loader, args, batch_i):
    
    # tSNEの可視化結果を保存する用のパスを作成
    logdir = os.path.join(args.tsne_dir, args.log_name)
    #os.makedirs(logdir, exist_ok=True)
    tsne_path = logdir
    
    model.eval()
    
    with torch.no_grad():
        
        # バッファ内のインデックスを獲得
        buff_index = train_loader.batch_sampler.fetch_buffer_data()
    
        if len(buff_index) <= 0:
            return
        
        # 可視化したい特徴量をランダムで選択
        #k = 50   # データ数
        #buff_index = random.sample(buff_index, k=k)
        
        
        
        # バッファ内の画像とラベル，データストリーム内のデータとラベルを獲得
        buff_images = []
        buff_labels = []
        for i in buff_index:
            buff_image, buff_label = train_loader.dataset.get_buffer_data(i)
            buff_images.append(buff_image)
            buff_labels.append(buff_label)   
        
        
        # データとラベルをそれぞれ連結
        #buff_images = torch.cat(buff_images, dim=0)
        #buff_images = torch.stack(buff_images, dim=0)
        buff_labels = torch.stack(buff_labels, dim=0)
        buff_labels_list = buff_labels.tolist()
        buff_labels_set = set(buff_labels_list)
        
        #print("buff_images.shape : ", buff_images.shape)
        
        # 特徴埋め込みを獲得
        # out of memoryを回避するために複数回に分けて実行
        features = []
        for buff_image in buff_images:
            #print("buff_image.shape : ", buff_image.shape)
            if torch.cuda.is_available():
                buff_image = buff_image.cuda()
                feature, _, _ = model(buff_image)
                features.append(feature)
        
        #features, _, _ = model(buff_images)
        #print("features.shape : ", features.shape)
        
        features = torch.cat(features, dim=0)
        #print("features.shape : ", features.shape)
        features = chunk_avg(features, args.num_patch)
        #print("features.shape : ", features.shape)
        
        
        features = features.detach().cpu().numpy()
        
        # tSNEモデルの作成と次元削減
        X = umap.UMAP(n_components=2).fit_transform(features)
        
        #print("len(buff_X_tsne) : ", len(buff_X_tsne))
        #print("len(stream_X_tsne) : ", len(stream_X_tsne))
        
        
        # 可視化
        for cls in buff_labels_set:
            idxs = [i for i, label in enumerate(buff_labels_list) if label == cls]
            plt.scatter(X[idxs, 0], X[idxs, 1], label=f"Class {cls}", marker="x", s=10)
        plt.legend()
        plt.title(f"t-SNE Visualization of {args.data_type} buffer Features")
        plt.xlabel("t-SNE Feature 1")
        plt.ylabel("t-SNE Feature 2")
        plt.show()
        plt.savefig(f'{tsne_path}/empssl_{args.data_type}_buffonly_{batch_i}_tsne.png', dpi=100, bbox_inches='tight')
        plt.close()
        
    
    model.train()